In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import random
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse

# Excel dosyası yükleme(jupyter için)
df = pd.read_excel("matchh.xlsx")  # "matchh.xlsx" dosyasını okuyarak bir DataFrame'e yükler

# ChromeDriver kurulumunu başlatalım
service = Service(ChromeDriverManager().install())  # ChromeDriver'ı otomatik olarak indirir ve hizmet olarak başlatır
driver = webdriver.Chrome(service=service)  # WebDriver ile Chrome tarayıcısını başlatır

# Ürün bağlantılarını saklayacak yeni bir sütun oluşturalım
df["product_link"] = None  # "product_Link" adlı yeni bir sütun ekler, başta tüm değerler None olarak ayarlanır

# Her bir başlık için arama yapacağız
for index, row in df.iterrows():  # DataFrame'deki her satır için döngü başlatır
    product_title = row['Ürün İsmi']  # 'Ürün İsmi' sütunundaki değeri alır
    
    # Arama URL'sini oluştur
    search_url = f"ürünaramauzantısı={urllib.parse.quote(product_title)}"  # Ürün adını URL'ye uygun hale getirip arama URL'sini oluşturur
    driver.get(search_url)  # linkte oluşturulan arama URL'sine gider
    
    # Sayfanın yüklenmesi için rastgele bekleme süresi (2 ila 5 saniye arasında)
    time.sleep(random.uniform(2, 5))  # Sayfanın tam yüklenmesini beklemek için rastgele bir süre bekler (2 ila 5 saniye arası)
    
    try:
        # Sayfada aşağı kaydırma işlemi
        scroll_count = random.randint(1, 3)  # Sayfa içinde rastgele kaydırma sayısını (1 ila 3) belirler
        for _ in range(scroll_count):  # Belirlenen sayıda kaydırma işlemi yapar
            driver.execute_script("window.scrollBy(0, 500);")  # 500 piksel aşağı kaydırır
            time.sleep(random.uniform(1, 3))  # Her kaydırma arasında 1 ila 3 saniye rastgele bekler
        
        # İlk sonucu bul ve bağlantıyı al
        first_result = driver.find_element(By.CSS_SELECTOR, ".prdct-cntnr-wrppr .p-card-wrppr a")  # CSS seçiciyi kullanarak ilk ürünü bulur
        product_link = first_result.get_attribute("href")  # İlk ürünün bağlantı (href) değerini alır
        df.at[index, "product_link"] = product_link  # Bağlantıyı DataFrame'deki ilgili hücreye ekler
    except Exception as e:
        print(f"Ürün bulunamadı veya hata oluştu: {product_title}")  # Ürün bulunamazsa veya hata oluşursa uyarı mesajı verir
        df.at[index, "product_link"] = "Link bulunamadı"  # Hata durumunda hücreye "Link bulunamadı" yazılır
    
    # Her arama arasında rastgele bekleme süresi (3 ila 6 saniye arasında)
    time.sleep(random.uniform(3, 6))  # Her ürün araması arasında rastgele 3 ila 6 saniye bekler

# Tarayıcıyı kapatalım
driver.quit()  # Tarayıcıyı kapatır

# Sonucu yeni dosyaya kaydedelim
output_file_path = "matchh.xlsx"  # Kayıt yeri olarak betiğin olduğu dizini kullanır
df.to_excel(output_file_path, index=False)  # DataFrame'i Excel dosyasına kaydeder, satır indeksini dahil etmez

print(f"Tamamlandı! Dosya kaydedildi: {output_file_path}")  # Tamamlandığını ve dosyanın kaydedildiğini belirtir
